# Linear Regression in statsmodels (Practice)
---

Author: Jackson Muehlbauer

Date: 3/19/23

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline, Pipeline

# metrics
from sklearn.metrics import r2_score, mean_squared_error

# statsmodel
import statsmodels.api as sm


In [2]:
# load data
path = 'archive/CarPrice_Assignment.csv'
df = pd.read_csv(path)
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


## Exploring and Cleaning DataFrame

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

No missing data

In [7]:
df_ob = df.select_dtypes(include = 'object')
df_ob.head()
for col in df_ob.columns:
    print(f'{col} \n {df_ob[col].value_counts()} \n')

CarName 
 toyota corona           6
toyota corolla          6
peugeot 504             6
subaru dl               4
mitsubishi mirage g4    3
                       ..
mazda glc 4             1
mazda rx2 coupe         1
maxda glc deluxe        1
maxda rx3               1
volvo 246               1
Name: CarName, Length: 147, dtype: int64 

fueltype 
 gas       185
diesel     20
Name: fueltype, dtype: int64 

aspiration 
 std      168
turbo     37
Name: aspiration, dtype: int64 

doornumber 
 four    115
two      90
Name: doornumber, dtype: int64 

carbody 
 sedan          96
hatchback      70
wagon          25
hardtop         8
convertible     6
Name: carbody, dtype: int64 

drivewheel 
 fwd    120
rwd     76
4wd      9
Name: drivewheel, dtype: int64 

enginelocation 
 front    202
rear       3
Name: enginelocation, dtype: int64 

enginetype 
 ohc      148
ohcf      15
ohcv      13
dohc      12
l         12
rotor      4
dohcv      1
Name: enginetype, dtype: int64 

cylindernumber 
 four  

CarName won't be particularly useful for modeling

In [9]:
df.duplicated().sum()

0

In [10]:
df['symboling'].value_counts()

 0    67
 1    54
 2    32
 3    27
-1    22
-2     3
Name: symboling, dtype: int64

No duplicates

## Initial Preprocessing

In [15]:
# validation split
X = df.drop(columns = ['car_ID', 'CarName', 'price']).copy()
y = df['price'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [33]:
# numeric columns don't need to be processed
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')
num_cols = num_selector(X_train) # saving these for later
cat_cols = cat_selector(X_train) # saving these for later


ohe = OneHotEncoder(handle_unknown='ignore')


cat_tup = (ohe, cat_selector)

preprocessor = make_column_transformer(cat_tup, remainder = 'passthrough')


In [34]:
# fit
preprocessor.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x14094bcd0>)])

In [38]:
df_temp = pd.DataFrame(preprocessor.transform(X_train))
df_temp.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,54.5,2017.0,103.0,2.99,3.47,21.9,55.0,4800.0,45.0,50.0
1,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,54.9,2326.0,122.0,3.31,3.54,8.7,92.0,4200.0,29.0,34.0
2,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,53.5,2024.0,97.0,3.15,3.29,9.4,69.0,5200.0,31.0,37.0
3,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,53.1,2507.0,136.0,3.19,3.40,8.5,110.0,5500.0,19.0,25.0
4,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,52.0,2540.0,146.0,3.62,3.50,9.3,116.0,4800.0,24.0,30.0


In [35]:
# get ohe step
ohe_step = preprocessor.named_transformers_['onehotencoder']

In [36]:
# get cat feature names
cat_features = ohe_step.get_feature_names_out(cat_cols)

In [40]:
# final feature list

final_features = []
final_features.extend(cat_features)
final_features.extend(num_cols)
final_features

['fueltype_diesel',
 'fueltype_gas',
 'aspiration_std',
 'aspiration_turbo',
 'doornumber_four',
 'doornumber_two',
 'carbody_convertible',
 'carbody_hardtop',
 'carbody_hatchback',
 'carbody_sedan',
 'carbody_wagon',
 'drivewheel_4wd',
 'drivewheel_fwd',
 'drivewheel_rwd',
 'enginelocation_front',
 'enginelocation_rear',
 'enginetype_dohc',
 'enginetype_dohcv',
 'enginetype_l',
 'enginetype_ohc',
 'enginetype_ohcf',
 'enginetype_ohcv',
 'enginetype_rotor',
 'cylindernumber_eight',
 'cylindernumber_five',
 'cylindernumber_four',
 'cylindernumber_six',
 'cylindernumber_twelve',
 'cylindernumber_two',
 'fuelsystem_1bbl',
 'fuelsystem_2bbl',
 'fuelsystem_4bbl',
 'fuelsystem_idi',
 'fuelsystem_mpfi',
 'fuelsystem_spdi',
 'fuelsystem_spfi',
 'symboling',
 'wheelbase',
 'carlength',
 'carwidth',
 'carheight',
 'curbweight',
 'enginesize',
 'boreratio',
 'stroke',
 'compressionratio',
 'horsepower',
 'peakrpm',
 'citympg',
 'highwaympg']

In [43]:
# make dataframe with feature names
X_train_df = pd.DataFrame(preprocessor.transform(X_train), columns = final_features, index = X_train.index)
X_test_df = pd.DataFrame(preprocessor.transform(X_test), columns = final_features, index = X_test.index)
X_train_df.head()

,fueltype_diesel,fueltype_gas,aspiration_std,aspiration_turbo,doornumber_four,doornumber_two,carbody_convertible,carbody_hardtop,carbody_hatchback,carbody_sedan,...,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg
90,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,54.5,2017.0,103.0,2.99,3.47,21.9,55.0,4800.0,45.0,50.0
173,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,54.9,2326.0,122.0,3.31,3.54,8.7,92.0,4200.0,29.0,34.0
93,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,53.5,2024.0,97.0,3.15,3.29,9.4,69.0,5200.0,31.0,37.0
5,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,53.1,2507.0,136.0,3.19,3.40,8.5,110.0,5500.0,19.0,25.0
167,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,52.0,2540.0,146.0,3.62,3.50,9.3,116.0,4800.0,24.0,30.0


In [44]:
## overwrite dfs to have constants
X_train_df = sm.add_constant(X_train_df, prepend = False, has_constant = 'add')
X_test_df = sm.add_constant(X_test_df, prepend = False, has_constant = 'add')
display(X_train_df.head(2), X_test_df.head(2))

,fueltype_diesel,fueltype_gas,aspiration_std,aspiration_turbo,doornumber_four,doornumber_two,carbody_convertible,carbody_hardtop,carbody_hatchback,carbody_sedan,...,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,const
90,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,2017.0,103.0,2.99,3.47,21.9,55.0,4800.0,45.0,50.0,1.0
173,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,2326.0,122.0,3.31,3.54,8.7,92.0,4200.0,29.0,34.0,1.0


,fueltype_diesel,fueltype_gas,aspiration_std,aspiration_turbo,doornumber_four,doornumber_two,carbody_convertible,carbody_hardtop,carbody_hatchback,carbody_sedan,...,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,const
15,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,3230.0,209.0,3.62,3.39,8.0,182.0,5400.0,16.0,22.0,1.0
9,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,3053.0,131.0,3.13,3.40,7.0,160.0,5500.0,16.0,22.0,1.0


## OLS statsmodels

In [46]:
# Getting summary
model = sm.OLS(y_train, X_train_df, hasconst = True)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                     55.20
Date:                Sun, 19 Mar 2023   Prob (F-statistic):           1.78e-57
Time:                        18:20:01   Log-Likelihood:                -1360.4
No. Observations:                 153   AIC:                             2801.
Df Residuals:                     113   BIC:                             2922.
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
fueltype_diesel       -2754.0108   3061.925     -0.899      0.370   -8820.237    3312.215
fueltype_gas          -6762.3873   3036.682     -2.227      0.028   -1.28e+04    -746.173
aspiration_std        -5830.9890   2041.671     -2.856      0.005   -9875.907   -1786.071
aspiration_turbo      -3685.4091   1914.199     -1.925      0.057   -7477.782     106.964
doornumber_four       -4661.2975   1993.045     -2.339      0.021   -8609.880    -712.715
doornumber_two        -4855.1005   1909.915     -2.542      0.012   -8638.988   -1071.214
carbody_convertible     391.1172   1061.363      0.369      0.713   -1711.634    2493.869
carbody_hardtop       -3386.3942   1316.419     -2.572      0.011   -5994.457    -778.331
carbody_hatchback     -2189.8314    996.073     -2.198      0.030   -4163.232    -216.431
carbody_sedan         -1341.0731   1010.567     -1.327      0.187   -3343.190     661.043
carbody_wagon         -2990.2165   1173.725     -2.548      0.012   -5315.577    -664.856
drivewheel_4wd        -3033.9503   1434.421     -2.115      0.037   -5875.797    -192.104
drivewheel_fwd        -3589.9257   1541.345     -2.329      0.022   -6643.609    -536.242
drivewheel_rwd        -2892.5221   1476.047     -1.960      0.053   -5816.837      31.793
enginelocation_front  -9870.6333   2400.932     -4.111      0.000   -1.46e+04   -5113.954
enginelocation_rear     354.2353   2484.564      0.143      0.887   -4568.134    5276.604
enginetype_dohc        -267.6125   1201.258     -0.223      0.824   -2647.522    2112.297
enginetype_dohcv      -6841.5974   4325.556     -1.582      0.117   -1.54e+04    1728.109
enginetype_l          -1209.6804   2051.071     -0.590      0.557   -5273.222    2853.861
enginetype_ohc         1988.0034   1255.997      1.583      0.116    -500.352    4476.359
enginetype_ohcf        -284.2699   1740.709     -0.163      0.871   -3732.929    3164.389
enginetype_ohcv       -5304.6838   1245.289     -4.260      0.000   -7771.825   -2837.542
enginetype_rotor       2403.4425   1519.008      1.582      0.116    -605.986    5412.871
cylindernumber_eight   5132.6433   2153.981      2.383      0.019     865.218    9400.069
cylindernumber_five   -2912.4014   1934.432     -1.506      0.135   -6744.861     920.058
cylindernumber_four   -3951.9226   1889.604     -2.091      0.039   -7695.568    -208.277
cylindernumber_six    -2098.7982   1007.476     -2.083      0.039   -4094.789    -102.808
cylindernumber_twelve -8089.3617   4412.912     -1.833      0.069   -1.68e+04     653.413
cylindernumber_two     2403.4425   1519.008      1.582      0.116    -605.986    5412.871
fuelsystem_1bbl        -657.5227   1072.036     -0.613      0.541   -2781.420    1466.375
fuelsystem_2bbl        -544.6957    830.758     -0.656      0.513   -2190.576    1101.185
fuelsystem_4bbl       -

In [47]:
# evaluate model on test data
test_preds = result.predict(X_test_df)

print(f'Testing r-squared: {r2_score(y_test, test_preds):.3f}')
print(f'Testing RMSE: {np.sqrt(mean_squared_error(y_test, test_preds)):.3f}')

Testing r-squared: 0.881
Testing RMSE: 2842.345
